In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from nltk import ngrams
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from math import sqrt
import re

In [2]:
train_df = pd.read_csv("data/training/data_train.csv", dtype='unicode')
train_df.fillna(value='', inplace=True)